# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
product_customer = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum()
product_customer.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
matrix = pd.pivot_table(product_customer, values='Quantity', index='CustomerID', columns='ProductName',fill_value=0)
matrix.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
square = 1/(1 + squareform(pdist(matrix, 'euclidean')))
square[:4]

array([[1.        , 0.0774211 , 0.08704668, ..., 0.00435486, 0.00416667,
        0.00433325],
       [0.0774211 , 1.        , 0.07844774, ..., 0.0043666 , 0.00416601,
        0.00433456],
       [0.08704668, 0.07844774, 1.        , ..., 0.00436527, 0.00417944,
        0.00433325],
       [0.08179978, 0.0764348 , 0.08006966, ..., 0.00436493, 0.00417548,
        0.00433906]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
distances = pd.DataFrame(data=square, index=matrix.index, columns=matrix.index)
distances.head()
similarities = distances[649].sort_values(ascending=False)[:6]
similarities

CustomerID
649     1.000000
3535    0.087410
2617    0.086333
639     0.085983
2503    0.085638
3317    0.085297
Name: 649, dtype: float64

In [20]:
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
product_customer.reset_index(drop=False, inplace=True)
product_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [9]:
selec_customers = product_customer[product_customer.CustomerID.isin(similarities.index)]
selec_customers.head()

,CustomerID,ProductName,Quantity
452,639,Anchovy Paste - 56 G Tube,1
453,639,Appetizer - Mushroom Tart,1
454,639,Assorted Desserts,1
455,639,Bread - Italian Roll With Herbs,1
456,639,Bread - Raisin Walnut Oval,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
degressif_product = selec_customers[['ProductName','Quantity']].groupby('ProductName').sum().sort_values('Quantity', ascending = False)
degressif_product.head()

,Quantity
ProductName,
Halibut - Steaks,5
Yoghurt Tubes,4
Soupfoamcont12oz 112con,4
Fenngreek Seed,4
Ketchup - Tomato,4


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
products_zero = degressif_product.merge(matrix.T[similarities.index], how='inner', on='ProductName')
prod_a_recommander = pd.Series(products_zero[649][products_zero[649] == 0].index)
prod_a_recommander.head()

0                Lamb - Ground
1    Appetizer - Mushroom Tart
2            Butter - Unsalted
3           Peas - Pigeon, Dry
4        Pepper - Black, Whole
Name: ProductName, dtype: object

In [12]:
products_zero = degressif_product.merge(matrix.T[similarities.index], how='inner', on='ProductName')
prod_a_recommander = pd.DataFrame(columns=[similarities.index])
for col in products_zero.columns:
    prod_a_recommander[col] = pd.Series(products_zero[col][products_zero[col] == 0].index)
prod_a_recommander.head()

CustomerID,649,3535,2617,639,2503,3317,Quantity
0,Lamb - Ground,Fenngreek Seed,Yoghurt Tubes,Yoghurt Tubes,Halibut - Steaks,Halibut - Steaks,NaN
1,Appetizer - Mushroom Tart,"Mushrooms - Black, Dried",Ketchup - Tomato,Soupfoamcont12oz 112con,Soupfoamcont12oz 112con,Yoghurt Tubes,NaN
2,Butter - Unsalted,Juice - V8 Splash,Olive - Spread Tapenade,"Mushrooms - Black, Dried",Fenngreek Seed,Ketchup - Tomato,NaN
3,"Peas - Pigeon, Dry",Appetizer - Mushroom Tart,Butter - Unsalted,Jagermeister,"Mushrooms - Black, Dried","Mushrooms - Black, Dried",NaN
4,"Pepper - Black, Whole",Olive - Spread Tapenade,Pastry - Choclate Baked,Lamb - Ground,Jagermeister,Jagermeister,NaN


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
product_customer = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum()
product_customer.reset_index(drop=False, inplace=True)
square = 1/(1 + squareform(pdist(matrix, 'euclidean')))
distances = pd.DataFrame(data=square, index=matrix.index, columns=matrix.index)
prod_recommande = pd.DataFrame(columns=[distances.index])
for col in distances.columns:
    similarities = distances[col].sort_values(ascending=False)[:6]
    selec_customers = product_customer[product_customer.CustomerID.isin(similarities.index)]
    degressif_product = selec_customers[['ProductName','Quantity']].groupby('ProductName').sum().sort_values('Quantity', ascending = False)
    products_zero = degressif_product.merge(matrix.T[similarities.index], how='inner', on='ProductName')
    prod_recommande[col] = pd.Series(products_zero[col][products_zero[col] == 0].index)
prod_recommande.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Soup - Campbells Bean Medley,Soup - Campbells Bean Medley,Soupfoamcont12oz 112con,"Veal - Inside, Choice","Sauce - Gravy, Au Jus, Mix",Butter - Unsalted,"Veal - Inside, Choice",Wine - Ej Gallo Sierra Valley,Lamb - Ground,"Veal - Inside, Choice",...,Cheese - Wine,Muffin Batt - Choc Chk,"Pork - Loin, Bone - In",Grenadine,"Placemat - Scallop, White","Soup - Campbells, Lentil",Skirt - 29 Foot,"Lamb - Whole, Fresh",Crackers - Trio,Beans - Wax
1,Butter - Unsalted,Wanton Wrap,Bread - Italian Roll With Herbs,Beef - Top Sirloin,Soup - Campbells Bean Medley,Sausage - Breakfast,Olive - Spread Tapenade,Cod - Black Whole Fillet,Appetizer - Mushroom Tart,Sardines,...,Muffin - Carrot Individual Wrap,Papayas,Potatoes - Idaho 100 Count,Fenngreek Seed,Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Milk - 1%,Chips Potato Salt Vinegar 43g,Pernod,Sherry - Dry
2,Wine - Blue Nun Qualitatswein,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Butter - Unsalted,Cheese - Cambozola,"Sauce - Gravy, Au Jus, Mix",Cheese - Cambozola,Soupfoamcont12oz 112con,Butter - Unsalted,Wine - Blue Nun Qualitatswein,...,Meldea Green Tea Liquor,Meldea Green Tea Liquor,Pants Custom Dry Clean,Halibut - Fletches,Tofu - Firm,Pants Custom Dry Clean,Beans - Kidney White,Tea - English Breakfast,Tea - Jasmin Green,Pork - Hock And Feet Attached
3,Wine - Ej Gallo Sierra Valley,Pork - Kidney,"Fish - Scallops, Cold Smoked",Lamb - Ground,Olive - Spread Tapenade,"Veal - Inside, Choice",Sprouts - Baby Pea Tendrils,Wine - Crozes Hermitage E.,"Peas - Pigeon, Dry",Wine - Ej Gallo Sierra Valley,...,"Lentils - Red, Dry",Mussels - Frozen,Cake - Box Window 10x10x2.5,Ice Cream Bar - Oreo Cone,Sobe - Tropical Energy,Wiberg Super Cure,Cheese - Taleggio D.o.p.,Bread - Raisin Walnut Oval,Lettuce - Spring Mix,Ecolab - Mikroklene 4/4 L
4,Steam Pan - Half Size Deep,Bay Leaf,Fondant - Icing,"Tart Shells - Sweet, 4",Sprouts - Baby Pea Tendrils,"Pepper - Black, Whole","Pepper - Black, Whole","Wine - Magnotta, Merlot Sr Vqa","Pepper - Black, Whole","Pepper - Black, Whole",...,Coffee - Dark Roast,Wine - Chablis 2003 Champs,Rum - Mount Gay Eclipes,"Wine - Red, Colio Cabernet","Cheese - Boursin, Garlic / Herbs",Shrimp - 31/40,Sprouts - Baby Pea Tendrils,Curry Paste - Madras,Pastry - Choclate Baked,Pears - Bosc


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [83]:
prod_recommande.head().T

,0,1,2,3,4
CustomerID,,,,,
33,Soup - Campbells Bean Medley,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Wine - Ej Gallo Sierra Valley,Steam Pan - Half Size Deep
200,Soup - Campbells Bean Medley,Wanton Wrap,Muffin - Carrot Individual Wrap,Pork - Kidney,Bay Leaf
264,Soupfoamcont12oz 112con,Bread - Italian Roll With Herbs,Wine - Two Oceans Cabernet,"Fish - Scallops, Cold Smoked",Fondant - Icing
356,"Veal - Inside, Choice",Beef - Top Sirloin,Butter - Unsalted,Lamb - Ground,"Tart Shells - Sweet, 4"
412,"Sauce - Gravy, Au Jus, Mix",Soup - Campbells Bean Medley,Cheese - Cambozola,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils
464,Butter - Unsalted,Sausage - Breakfast,"Sauce - Gravy, Au Jus, Mix","Veal - Inside, Choice","Pepper - Black, Whole"
477,"Veal - Inside, Choice",Olive - Spread Tapenade,Cheese - Cambozola,Sprouts - Baby Pea Tendrils,"Pepper - Black, Whole"
639,Wine - Ej Gallo Sierra Valley,Cod - Black Whole Fillet,Soupfoamcont12oz 112con,Wine - Crozes Hermitage E.,"Wine - Magnotta, Merlot Sr Vqa"
649,Lamb - Ground,Appetizer - Mushroom Tart,Butter - Unsalted,"Peas - Pigeon, Dry","Pepper - Black, Whole"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [18]:
product_customer = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum()
product_customer.reset_index(drop=False, inplace=True)
square = 1/(1 + squareform(pdist(matrix, 'cosine')))
distances = pd.DataFrame(data=square, index=matrix.index, columns=matrix.index)
prod_recommande = pd.DataFrame(columns=[distances.index])
for col in distances.columns:
    similarities = distances[col].sort_values(ascending=False)[:6]
    selec_customers = product_customer[product_customer.CustomerID.isin(similarities.index)]
    degressif_product = selec_customers[['ProductName','Quantity']].groupby('ProductName').sum().sort_values('Quantity', ascending = False)
    products_zero = degressif_product.merge(matrix.T[similarities.index], how='inner', on='ProductName')
    prod_recommande[col] = pd.Series(products_zero[col][products_zero[col] == 0].index)
prod_recommande[:5].T.head()

,0,1,2,3,4
CustomerID,,,,,
33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Banana Turning
200,Snapple Lemon Tea,Longos - Grilled Salmon With Bbq,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,Wiberg Super Cure
412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Sea Bass - Whole


In [19]:
product_customer = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum()
product_customer.reset_index(drop=False, inplace=True)
square = 1/(1 + squareform(pdist(matrix, 'correlation')))
distances = pd.DataFrame(data=square, index=matrix.index, columns=matrix.index)
prod_recommande = pd.DataFrame(columns=[distances.index])
for col in distances.columns:
    similarities = distances[col].sort_values(ascending=False)[:6]
    selec_customers = product_customer[product_customer.CustomerID.isin(similarities.index)]
    degressif_product = selec_customers[['ProductName','Quantity']].groupby('ProductName').sum().sort_values('Quantity', ascending = False)
    products_zero = degressif_product.merge(matrix.T[similarities.index], how='inner', on='ProductName')
    prod_recommande[col] = pd.Series(products_zero[col][products_zero[col] == 0].index)
prod_recommande[:5].T.head()

,0,1,2,3,4
CustomerID,,,,,
33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco
200,Otomegusa Dashi Konbu,Potatoes - Idaho 100 Count,Milk Powder,Pail With Metal Handle 16l White,Crackers - Trio
264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Pickerel - Fillets,Garbag Bags - Black
356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Cheese - Cheddarsliced,Ocean Spray - Kiwi Strawberry,Olives - Kalamata
412,Cake - Mini Cheesecake,Butter - Unsalted,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,"Beans - Kidney, Red Dry"
